### 본 예제는 tensorflow 내에서 제공하는 파일 입출력 기능을 활용하는 것을 다룹니다.

Tensorflow 내에서 데이터를 읽는 방법은 크게 3가지가 있습니다.
* Constant를 통해 읽기: constant 형식의 텐서를 선언하여 그대로 사용하는 경우를 의미합니다.
* Feed dictionary 이용: 위의 방식이나 다른 방식 등으로 읽은 데이터를 feed_dict를 통하여 모델에 입력합니다.
* Reader 이용: tensorflow 내에서 제공하고 있는 reader를 이용하는 방법입니다.

constant를 통한 방법, feed dictionary를 이용한 방법은 이 글을 읽으시는 분이라면 모두 한 번 씩은 사용해보셨을 것입니다. 

사용법이 직관적이며 쉽다는 점에서 많은 사람들이 유용하게 사용하고 있는데요. 

항상 이 방법으로 데이터를 읽는 것이 가능한 것이 아닙니다. 

예를 들어, 우리가 다루는 데이터의 용량이 매우 크다고 생각해볼게요. 

파일을 읽어서, 대규모의 데이터를 모두 메모리에 올리게 되면, 메모리 부족 현상이 일어나며 우리의 프로그램을 사용하지 못할 수도 있습니다.

반면, 우리가 딥러닝 모델을 학습할 때는 모든 데이터를 한 번에 사용하지는 않습니다. 

모델 학습 시 (또는 계산 시)에 한 번에 몇 개의 데이터를 사용하는 지의 단위를 우리는 batch라고 부르며, 해당 데이터의 크기를 batch_size라고 말합니다.

즉, mini-batch learning에서는 한 번 계산 시에 batch_size 개수 만큼의 데이터가 필요하지, 전체 데이터를 읽어서 가지고 있을 필요는 없습니다. 

Tensorflow 내에서 제공하는 Queue 기능과 Reader 기능을 이용하면 이를 손쉽게 구현할 수 있습니다.

즉, 우리가 필요한 데이터의 개수만큼 (stream으로) 파일에서 불러와 사용하는 것을 가능하게 해줍니다.

심지어 multi threading 또한 제공하며, 이를 관리하기 위한 tf.train.Coordinaotr를 제공해줍니다.

본 예제는 tensorflow에서 제공하고 있는 queue와 reader를 통하여 파일을 읽는 방법을 다룹니다.

### 1. Queue and Threading

이번 예제에서 다루고자 하는 목표는 사실 queue나 multi-threading을 자유롭게 쓰는 것보다, tensorflow 내에서 제공해주는 다양한 reader를 사용하는 것인데요.

해당 reader 들이 기본적으로 queue, threading 구조를 가지고 구현이 되어 있어, 우선 tensorflow에서 이를 어떻게 사용하는지 알아봅시다.

아래는 Tensorflow 내에서 사용되는 Queue와 Multi-Threading과 관련된 기본적인 예제입니다.

#### 1.1 데이터 생성

우선 queue에 집어넣을 임의의 데이터를 만들어보겠습니다. 4개의 feature로 구성되어 있는 데이터가 0-1 label을 가지고 있다고 가정해봅시다.

우선 4개의 특징(feature)를 가지는 1000개의 데이터 포인트를 만들어보겠습니다. (x 값)
각 데이터 포인트는 0 또는 1의 분류 결과를 가지고 있다고 가정해볼게요. (y 값)

In [11]:
import numpy as np
import tensorflow as tf

N_SAMPLES = 1000

# Data Generation (1000 samples with normal distribution
data = 10*np.random.randn(N_SAMPLES, 4) + 1
#np.random.randn(# of data, features) : generate random standard normal distribution data

target = np.random.randint(0,2,size=N_SAMPLES)
#np.random.randint(min, max(not included), # of data) : generate random integer number

print "X data: ", data
print "Y data: ", target

 X data:  [[ 13.76131533   4.37361799   1.20334722  15.20080039]
 [ -4.87622575   4.04066957   2.67448328   7.82210814]
 [  4.62439362  -8.22727783  -4.96933855  15.1395118 ]
 ..., 
 [ 13.02104794   8.16428606   1.79281911   2.20602291]
 [ -7.73458255  -5.16380294   2.97646644  10.95263377]
 [  4.2194718    3.62759029  13.2138805    5.54566171]]
Y data:  [1 1 1 0 1 1 1 1 0 0 0 1 0 0 0 0 0 1 0 0 1 1 0 0 1 1 1 0 1 0 0 1 1 1 0 0 1
 0 0 1 1 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 1 1 0 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0
 0 0 1 1 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1
 0 1 0 0 0 0 1 1 0 0 1 1 0 1 1 1 1 1 0 1 0 0 1 0 1 1 1 1 0 0 0 0 1 0 0 1 0
 0 0 0 0 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0 0 0 1 1 0 0
 0 1 0 1 1 1 1 1 0 0 1 1 1 1 1 0 0 0 1 0 0 0 0 1 1 1 1 1 0 1 1 0 1 0 0 1 1
 1 1 0 1 0 1 1 0 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1 0 0 0 1 0 1 1 1 0 0 0 0 0 0
 1 1 0 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 1 0 1
 0 1 0 0 0 0 0 1 1 0 1 1 1 0 1 1 0 0 1 0 0 

#### 1.2 Queue, Enqueue, Dequeue

이제 우리가 만든 데이터를 Queue에 집어 넣어 보겠습니다.
참고로 Queue에 집어 넣는 것을 enqueue, Queue에서 값을 가져오는 것을 dequeue 라고 부릅니다.

tensorflow에서는 다양한 queue class가 있지만 2가지 queue를 주로 사용합니다. (자세한 사항은 tensorflow.org 참조해주세요 ^^)

* tf.FIFOQueue : FIFO (First-In, First-Out) 방식으로 구현된 Queue. 먼저 들어온 친구가 먼저 나가는 규칙
* tf.RandomShuffleQueue : 들어온 순서에 관계 없이, 랜덤한 순서로 값을 내보내는 Queue

위의 구현되어 있는 Queue들은 값을 enqueue, dequeue 하기 위해 3가지 기능을 이용합니다.
* enqueue : 우리가 설정한 Queue에 단일 element를 집어 넣습니다.
* enqueue_many : 우리가 설정한 Queue에 여러 elements를 집어 넣습니다.
* dequeue : 우리가 설정한 Queue에서 단일 element를 받아옵니다. 

이 외에도 dequeue_many, dequeue_upto 등의 함수들이 존재합니다. 하지만 기존 사용자들이 실행하며 에러가 많이 발생하여, 지원을 중단하였다가 v.1.14 부터 다시 제공하는 기능입니다.

여러 샘플을 dequeue하고 싶으실 때는 dequeue_many 보다 tf.train.batch를 추천드립니다. (아래서 다를 예정)

tf.FIFOQueue에서 입력받는 주요 argument는 아래와 같습니다.

* capacity: Queue에 들어갈 수 있는 샘플의 최대 값
* dtypes: Queue에 들어가는 샘플들의 데이터 타입 (여러 타입의 데이터가 한 샘플에 pair로 들어가면 list로 선언)
* shapes: 데이터 샘플의 shape (list)
* name: tensorflow 내에서 사용될 이름

queue를 선언해준 후, 이를 사용하기 위해서는 tf.train.QueueRunner를 선언하면 됩니다.

QueueRunner는 사용할 queue와 queue에 데이터를 집어 넣는 enqueue 오퍼레이션과 함께 생성해야 합니다.
(enqueue 오퍼레이션은 위에서 말한 enqueue, enqueue_many를 의미합니다.)

이렇게 Queue를 실행해줄 수 있는 준비가 완료되면 실행만 해보면 됩니다!!!! 면 좋겠지만

한 가지 처리를 해주어야 합니다.

multi-threading을 사용할 때 중요한 점은 각 threads가 데이터를 읽는 속도, 문제가 생겼을 때 끝나는 속도를 맞추어주어야 합니다. 그렇지 않으면 특정 thread는 다른 thread의 처리를 기다리게 되기도 하고, 어떤 thread에서 문제가 생겼는데 다른 thread는 이를 알아채지 못하고 계속 데이터를 읽어들일 수 있습니다.

이를 위해 tf.Coordinator를 선언하여, thread 생성 시 (QueueRunner.create_threads) 에 함께 사용해줍니다.

자세한 사항은 아래를 보겠습니다.

In [18]:
#Enqueue and Dequeue of Generated Data

NUM_THREADS = 4

queue = tf.FIFOQueue(capacity=50, dtypes=[tf.float32, tf.int32], shapes=[[4],[]])

enqueue_op = queue.enqueue_many([data, target])

#enqueue_op = queue.enqueue([data, target]) 
##If we use queue.enqueue, it reads all of the data

dequeue_op = queue.dequeue()

qr = tf.train.QueueRunner(queue, [enqueue_op] * NUM_THREADS)
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    for step in range(100):
        if coord.should_stop():
            break
        data_batch, label_batch = sess.run(dequeue_op)
        print step, "th batch \n"
        print data_batch, label_batch, '\n'
    coord.request_stop()
    coord.join(enqueue_threads)

0 th batch 

[ 13.76131535   4.37361813   1.20334721  15.20079994] 1 

1 th batch 

[-4.87622595  4.04066944  2.6744833   7.82210827] 1 

2 th batch 

[  4.62439346  -8.22727776  -4.96933842  15.13951206] 1 

3 th batch 

[ 18.46115112  18.10789108  -1.19017899 -10.96013927] 0 

4 th batch 

[ -3.86222577   5.73549509 -13.90797424  15.11517334] 1 

5 th batch 

[  4.90439749  17.86127663  -6.50209236   8.04216671] 1 

6 th batch 

[  4.88619852  -5.55098438  12.07865524  12.00930309] 1 

7 th batch 

[-17.87566376 -11.80900288  -5.84214211   7.73138523] 1 

8 th batch 

[  1.60436988   8.42853355 -12.68644714  21.56090164] 0 

9 th batch 

[ 7.01393938  0.68976951  9.1928854  -0.67326391] 0 

10 th batch 

[ -0.52620167  -5.78049946  12.51230717  -9.90974808] 0 

11 th batch 

[ 11.27158737  14.46955776  -2.97492123  -5.72536469] 1 

12 th batch 

[-8.33271694  2.46627355 -0.45775086  4.20743895] 0 

13 th batch 

[  2.24191006e-02  -2.29729710e+01   3.54341626e-01   5.05730009e+00] 0 

### 1.2 Data Readers

우리가 대규모의 데이터를 다루는 경우, numpy 등을 이용해 한번에 모든 데이터를 읽게되면 메모리 부족 현상이 발생할 수 있습니다.

반면, 우리가 학습이나 테스트를 위해 모든 데이터를 동시에 사용하는 경우는 극히 적습니다.

이 경우, Data Raders를 통하여 필요한 크기만큼 효율적으로 데이터를 지속적으로 불러들여오면서 메모리를 관리할 수 있습니다.

.


우선, 읽을 파일의 이름을 queue에 저장합니다.

그리고 해당 파일의 형식이나, 읽는 방식에 따라서 적절한 Reader를 선언한 후, Reader.read를 통해 읽어주면 됩니다.

주로 사용되는 Reader의 종류는 아래와 같습니다.

* tf.TextLineReader : text, csv 파일 등에서 새로운 1줄을 읽음

* tf.FixedLengthRecordReader : 파일들이 같은 길이를 가지고 있을 때, 파일 전체를 읽음

* tf.WholeFileReader : 특정 파일의 전체 내용을 읽음

* tf.TFRecordReader : TFRecord (tensorflow 만의 binary 형식) 파일을 읽음

* tf.ReaderBase : 사용자가 직접 reader를 만들도록 제공하는 추상 클래스

이 후, Reader.read(filename_queue)를 사용하게 되면, queue에 들어있는 파일을 Reader의 종류에 따라 읽게됩니다.

읽은 후에 두 가지를 반환합니다. (key, value)

key는 각 파일명과 읽은 위치 등의 정보를 포함하며, value는 실제 element 값을 포함합니다.

In [19]:
# Lab 4 Multi-variable linear regression
# https://www.tensorflow.org/programmers_guide/reading_data

import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

filename_queue = tf.train.string_input_producer(
    ['data-01-test-score.csv'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader() #Text를 Line 단위로 읽는 Reader
key, value = reader.read(filename_queue)

읽은 데이터는 모두 string 형식으로 되어 있습니다.

string 형식을 그대로 사용할 것이라면 문제 없지만, 다른 형식으로 사용해야 한다면 decode 기능을 통하여 변경해주어야 합니다.

In [20]:
# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

### record_defaults, decode_csv 설명

여기서 헷갈리는 부분이 record_defaults 입니다.

일단 텐서플로 내에서 우리가 지정한 파일을 1줄 읽었을 텐데요.

읽은 데이터를 실제 사용가능한 양식으로 decoding을 해줄 때, decoding할 데이터의 양식을 정해주어야합니다.

우리가 지금 예제에서 다루고 있는 데이터는 4차원의 float 형식을 따르고 있습니다.

예를 들어 (0.6, 0.7, 0.3, 1.5) 이런 식으로 구성되어 있습니다.

tensorflow 내에서 decode 해줄 때는 직접 decoding할 양식을 정해주어야 합니다.


이 때, record_defaults는 크게 두 가지 기능을 합니다. 

먼저, 우리가 decode 해야할 4가지의 값이 각각 어떤 형식으로 되어있는지 대략적으로 알려주는 것인데요.

모두 float이므로, 여기에서 record defaults의 0. 이 의미하는 것은

이 부분의 값은 0. (float) 과 같은 형식으로 이루어져있으니까 이런 식으로 decode 해줘~~~ 라고 설정하는 것입니다.

꼭 0.을 사용하실 필요는 없고, 1000. 이든, 100000. 이든 float 형식의 아무 값이나 지정해주시면 됩니다.

두 번째는, 결측된 값 (absent)을 어떻게 처리할 지를 결정합니다. 

즉, 어떤 데이터가 비어 있다면 우리가 지정한 record_defaults 값을 채워 넣습니다.

### tf.train.batch

우리가 reader로 데이터를 읽을 경우, tensorflow는 tf.train.batch라는 아주 유용한 기능을 제공해줍니다.

현재 우리가 사용하는 reader는 tf.TextLineReader로 Text 파일의 1줄을 읽는 것인데요.

전체 데이터를 읽으려면 전체 데이터 개수만큼 읽어야합니다.

딥러닝 모델 학습에서 주로 mini-batch 학습을 사용하고, 이 때 모든 데이터를 동시에 쓰지 않고 batch_size 만큼의 데이터만 필요합니다.

tf.train.batch([....], batch_size=..) 를 사용하게 되면, batch_size 횟수만큼 reader를 통해 데이터를 불러온 후 디코딩을 한 텐서를 반환하게 됩니다.

위의 코드에서는 앞의 3개를 x 값으로, 마지막 1개를 y 값으로 설정한 후, batch_size=10 으로 지정하였습니다.

즉, train_x_batch, train_y_batch는 각각 10개의 데이터가 텐서 형태로 반환되게 됩니다.

이 때 주의할 점은, 이 번 예제에서는 test의 간소화를 위해 feed_dict를 사용하고 있습니다.

feed_dict에는 실제 데이터가 들어가야하기 때문에 (텐서가 아닌), sess.run을 통해 train_x_batch와 train_y_batch를 직접 실행해주어야 합니다.

(placeholder를 사용하지 않을 경우, 직접 train_x_batch, train_y_batch를 모델에 연결하여 사용하면 됩니다.)

### 아래는 Logistic Regression 모델 그래프를 구축한 것입니다.

In [21]:
# collect batches of csv in
train_x_batch, train_y_batch = tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

In [24]:
# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

In [25]:
# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

coord.request_stop()
coord.join(threads)

# Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))

'''
Your score will be  [[ 177.78144836]]
Other scores will be  [[ 141.10997009]
 [ 191.17378235]]

'''

(0, 'Cost: ', 76278.18, '\nPrediction:\n', array([[-121.86820221],
       [-139.39080811],
       [-141.32637024],
       [-150.58866882],
       [-107.16174316],
       [ -72.7861557 ],
       [-109.56083679],
       [ -77.0738678 ],
       [-124.42765808],
       [-107.08682251]], dtype=float32))
(10, 'Cost: ', 20.561777, '\nPrediction:\n', array([[ 147.80361938],
       [ 184.7509613 ],
       [ 178.0447998 ],
       [ 197.23188782],
       [ 140.05352783],
       [ 109.47977448],
       [ 150.79074097],
       [ 115.92835236],
       [ 178.51217651],
       [ 173.51348877]], dtype=float32))
(20, 'Cost: ', 29.307699, '\nPrediction:\n', array([[ 149.64117432],
       [ 186.92903137],
       [ 180.20739746],
       [ 199.57765198],
       [ 141.71485901],
       [ 110.68644714],
       [ 152.54115295],
       [ 117.21923828],
       [ 180.52909851],
       [ 175.35803223]], dtype=float32))
(30, 'Cost: ', 29.182575, '\nPrediction:\n', array([[ 149.67121887],
       [ 186.93441772],
   

       [ 173.82090759]], dtype=float32))
(370, 'Cost: ', 23.063879, '\nPrediction:\n', array([[ 150.22972107],
       [ 186.64109802],
       [ 180.4618988 ],
       [ 199.54768372],
       [ 141.50111389],
       [ 109.93817139],
       [ 152.34086609],
       [ 116.85168457],
       [ 179.65361023],
       [ 173.77886963]], dtype=float32))
(380, 'Cost: ', 22.909994, '\nPrediction:\n', array([[ 150.24485779],
       [ 186.63316345],
       [ 180.46821594],
       [ 199.54647827],
       [ 141.49520874],
       [ 109.91828918],
       [ 152.33528137],
       [ 116.84174347],
       [ 179.63024902],
       [ 173.73701477]], dtype=float32))
(390, 'Cost: ', 22.757542, '\nPrediction:\n', array([[ 150.25993347],
       [ 186.6252594 ],
       [ 180.4744873 ],
       [ 199.54527283],
       [ 141.48934937],
       [ 109.89851379],
       [ 152.32971191],
       [ 116.83187103],
       [ 179.60699463],
       [ 173.69538879]], dtype=float32))
(400, 'Cost: ', 22.606243, '\nPrediction:\n', arra

(640, 'Cost: ', 19.346035, '\nPrediction:\n', array([[ 150.61480713],
       [ 186.43914795],
       [ 180.62245178],
       [ 199.51681519],
       [ 141.35144043],
       [ 109.43238068],
       [ 152.19810486],
       [ 116.59828949],
       [ 179.05912781],
       [ 172.71380615]], dtype=float32))
(650, 'Cost: ', 19.224413, '\nPrediction:\n', array([[ 150.62815857],
       [ 186.43215942],
       [ 180.6280365 ],
       [ 199.51573181],
       [ 141.3462677 ],
       [ 109.41483307],
       [ 152.19313049],
       [ 116.58946228],
       [ 179.03851318],
       [ 172.67683411]], dtype=float32))
(660, 'Cost: ', 19.103809, '\nPrediction:\n', array([[ 150.64146423],
       [ 186.42518616],
       [ 180.63357544],
       [ 199.5146637 ],
       [ 141.34109497],
       [ 109.39736176],
       [ 152.18818665],
       [ 116.58068085],
       [ 179.01795959],
       [ 172.64002991]], dtype=float32))
(670, 'Cost: ', 18.984259, '\nPrediction:\n', array([[ 150.6546936 ],
       [ 186.41824341

(920, 'Cost: ', 16.308426, '\nPrediction:\n', array([[ 150.96653748],
       [ 186.25502014],
       [ 180.76930237],
       [ 199.48841858],
       [ 141.21537781],
       [ 108.97026825],
       [ 152.06684875],
       [ 116.365242  ],
       [ 178.51600647],
       [ 171.74000549]], dtype=float32))
(930, 'Cost: ', 16.213001, '\nPrediction:\n', array([[ 150.97828674],
       [ 186.24888611],
       [ 180.7742157 ],
       [ 199.48748779],
       [ 141.21084595],
       [ 108.95484161],
       [ 152.06246948],
       [ 116.35743713],
       [ 178.49787903],
       [ 171.70750427]], dtype=float32))
(940, 'Cost: ', 16.11837, '\nPrediction:\n', array([[ 150.98997498],
       [ 186.24278259],
       [ 180.77909851],
       [ 199.48654175],
       [ 141.20635986],
       [ 108.93949127],
       [ 152.05810547],
       [ 116.34966278],
       [ 178.47984314],
       [ 171.67514038]], dtype=float32))
(950, 'Cost: ', 16.0245, '\nPrediction:\n', array([[ 151.00160217],
       [ 186.23667908],


(1270, 'Cost: ', 13.40976, '\nPrediction:\n', array([[ 151.34681702],
       [ 186.05638123],
       [ 180.92831421],
       [ 199.45750427],
       [ 141.06912231],
       [ 108.47052002],
       [ 151.92388916],
       [ 116.11120605],
       [ 177.92871094],
       [ 170.68598938]], dtype=float32))
(1280, 'Cost: ', 13.33901, '\nPrediction:\n', array([[ 151.35679626],
       [ 186.05117798],
       [ 180.93249512],
       [ 199.45669556],
       [ 141.06529236],
       [ 108.4573822 ],
       [ 151.92012024],
       [ 116.10449219],
       [ 177.9132843 ],
       [ 170.65827942]], dtype=float32))
(1290, 'Cost: ', 13.268911, '\nPrediction:\n', array([[ 151.366745  ],
       [ 186.04600525],
       [ 180.93666077],
       [ 199.45588684],
       [ 141.06149292],
       [ 108.44431305],
       [ 151.91636658],
       [ 116.09781647],
       [ 177.89793396],
       [ 170.63072205]], dtype=float32))
(1300, 'Cost: ', 13.19943, '\nPrediction:\n', array([[ 151.37666321],
       [ 186.0408477

(1540, 'Cost: ', 11.696388, '\nPrediction:\n', array([[ 151.60089111],
       [ 185.92401123],
       [ 181.03474426],
       [ 199.43670654],
       [ 140.97201538],
       [ 108.13651276],
       [ 151.8276062 ],
       [ 115.94000244],
       [ 177.53623962],
       [ 169.980896  ]], dtype=float32))
(1550, 'Cost: ', 11.640165, '\nPrediction:\n', array([[ 151.60971069],
       [ 185.91941833],
       [ 181.03845215],
       [ 199.43600464],
       [ 140.96865845],
       [ 108.12491608],
       [ 151.82424927],
       [ 115.934021  ],
       [ 177.52262878],
       [ 169.9564209 ]], dtype=float32))
(1560, 'Cost: ', 11.584419, '\nPrediction:\n', array([[ 151.6184845 ],
       [ 185.91485596],
       [ 181.04212952],
       [ 199.43527222],
       [ 140.96531677],
       [ 108.11338043],
       [ 151.82092285],
       [ 115.92809296],
       [ 177.50906372],
       [ 169.93206787]], dtype=float32))
(1570, 'Cost: ', 11.529101, '\nPrediction:\n', array([[ 151.62721252],
       [ 185.9103

(1810, 'Cost: ', 10.332924, '\nPrediction:\n', array([[ 151.82511902],
       [ 185.80741882],
       [ 181.12882996],
       [ 199.41822815],
       [ 140.88679504],
       [ 107.84162903],
       [ 151.74198914],
       [ 115.78767395],
       [ 177.18977356],
       [ 169.3578949 ]], dtype=float32))
(1820, 'Cost: ', 10.288103, '\nPrediction:\n', array([[ 151.832901  ],
       [ 185.8033905 ],
       [ 181.13209534],
       [ 199.41755676],
       [ 140.8838501 ],
       [ 107.83140564],
       [ 151.73899841],
       [ 115.78236389],
       [ 177.17774963],
       [ 169.33625793]], dtype=float32))
(1830, 'Cost: ', 10.243739, '\nPrediction:\n', array([[ 151.84065247],
       [ 185.79936218],
       [ 181.13536072],
       [ 199.41694641],
       [ 140.88092041],
       [ 107.82122803],
       [ 151.73603821],
       [ 115.77708435],
       [ 177.165802  ],
       [ 169.31474304]], dtype=float32))
(1840, 'Cost: ', 10.199644, '\nPrediction:\n', array([[ 151.8483429 ],
       [ 185.7953

'\nYour score will be  [[ 177.78144836]]\nOther scores will be  [[ 141.10997009]\n [ 191.17378235]]\n\n'